In [12]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

rot = [] 

# Path to your PDF
pdf_path = "1st Amendment.pdf"

pytesseract.pytesseract.tesseract_cmd = "C:\\Tesseract-OCR\\tesseract.exe"


# Convert PDF to images (one per page)
pages = convert_from_path(pdf_path,poppler_path="C:\\poppler-22.12.0\\Library\\bin")

# Function to detect orientation of an image
def detect_orientation(image):
    # Use psm 0 for orientation and script detection
    osd = pytesseract.image_to_osd(image, config='--psm 0')
    print(osd)
    # Parse the orientation angle from the output
    for line in osd.split("\n"):
        if "Orientation in degrees" in line:
            angle = int(line.split(":")[1].strip())
        if "Rotate" in line:
            rotate = int(line.split(":")[1].strip())
    return angle, rotate

# Detect orientation for each page

for i, page in enumerate(pages):
    angle, rotate = detect_orientation(page)
    rot.append(rotate)
    #print(f"Page {i+1}: Orientation={angle}°, Rotate={rotate}°")
print(rot)


Page number: 0
Orientation in degrees: 270
Rotate: 90
Orientation confidence: 13.19
Script: Cyrillic
Script confidence: 0.80

Page number: 0
Orientation in degrees: 270
Rotate: 90
Orientation confidence: 13.61
Script: Latin
Script confidence: 0.51

Page number: 0
Orientation in degrees: 270
Rotate: 90
Orientation confidence: 13.48
Script: Cyrillic
Script confidence: 1.13

Page number: 0
Orientation in degrees: 270
Rotate: 90
Orientation confidence: 9.78
Script: Latin
Script confidence: 0.63

[90, 90, 90, 90]


In [6]:
import ocrmypdf

In [7]:
output_pdf_path = "output.pdf"

In [8]:
ocrmypdf.ocr(
        pdf_path,
        output_pdf_path,
        language="eng",
        force_ocr=True,
    )


Output()

Output()

[tesseract] lots of diacritics - possibly poor OCR

[tesseract] lots of diacritics - possibly poor OCR

[tesseract] lots of diacritics - possibly poor OCR

...

Output()

Output()

Output()

The output file size is 2.51× larger than the input file.
Possible reasons for this include:
--force-ocr was issued, causing transcoding.
The optional dependency 'jbig2' was not found, so some image optimizations could not be attempted.
The optional dependency 'pngquant' was not found, so some image optimizations could not be attempted.
PDF/A conversion was enabled. (Try `--output-type pdf`.)



<ExitCode.ok: 0>

In [13]:
import os
from PyPDF2 import PdfReader, PdfWriter

# ---------- PART 1: ROTATE PDF PAGES ----------
input_pdf = pdf_path
rotated_pdf_path = "rotated.pdf"
 
reader = PdfReader(input_pdf)
writer = PdfWriter()
 
# Rotate each page by 90 degrees
for page, angle in zip(reader.pages, rot):
    page.rotate(angle)  # Rotates clockwise 90°
    writer.add_page(page)
 
# Save rotated PDF
with open(rotated_pdf_path, "wb") as output_pdf:
    writer.write(output_pdf)
 
print("Rotated PDF saved as:", rotated_pdf_path)

Rotated PDF saved as: rotated.pdf


In [2]:
osd

NameError: name 'osd' is not defined

In [ ]:
import subprocess
import re

# Path to your PDF
input_pdf = "1st Amendment.pdf"
output_pdf = r""  # Required by ocrmypdf, even if we skip text

# Run ocrmypdf with PSM-0 (orientation detection only) and verbose output
command = [
    "ocrmypdf",
    "--skip-text",                 # Don't re-OCR pages with text
    "--tesseract-pagesegmode", "0", # PSM 0 = orientation & script detection
    "-v",                          # Verbose output
    input_pdf,
    output_pdf
]

# Run the command and capture stdout/stderr
result = subprocess.run(command, capture_output=True, text=True)

# Parse verbose output for page rotation
rotation_info = []
for line in result.stdout.splitlines():
    match = re.search(r'Page (\d+) rotated (\d+) degrees', line)
    if match:
        page_num = int(match.group(1))
        rotated_deg = int(match.group(2))
        rotation_info.append((page_num, rotated_deg))

# Print detected orientations
for page, angle in rotation_info:
    print(f"Page {page}: Rotate {angle}°")
